In [1]:
import pandas as pd
import sklearn as sfs
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('..')
from data.dataset import DATASET as dt

In [2]:
dt.head()

,city,latitude,longitude,population,country_code,c40,risk0,risk1,risk2,risk3,...,SDG 6.4.1. Services Water Use Efficiency,SDG 6.4.1. Water Use Efficiency,SDG 6.4.2. Water Stress,Seasonal variability (WRI),Total internal renewable water resources per capita,Total population with access to safe drinking-water (JMP),Total renewable water resources per capita,Total water withdrawal per capita,Urban population with access to safe drinking-water (JMP),country
0,Aalborg,57.0337,9.9166,122219.0,DNK,False,NaN,NaN,NaN,NaN,...,558.335628,368.612902,20.040562,1.3,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
1,Aarhus,56.1572,10.2107,237551.0,DNK,False,NaN,NaN,NaN,NaN,...,558.335628,368.612902,20.040562,1.3,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
2,Copenhagen,55.6786,12.5635,1085000.0,DNK,False,NaN,2.0,NaN,2.0,...,558.335628,368.612902,20.040562,1.3,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
3,Esbjerg,55.4670,8.4500,72205.0,DNK,False,NaN,NaN,NaN,NaN,...,558.335628,368.612902,20.040562,1.3,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
4,Frederikshavn,57.4337,10.5333,24103.0,DNK,False,NaN,2.0,NaN,NaN,...,558.335628,368.612902,20.040562,1.3,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark


# Dataset splitting

In [3]:
import random

def data_splitting(dt,risk):
    # Select the columns containing labelled risk remove labels from the dataset to define training set
    train = dt[dt[risk].notnull()]
    y_train = train[risk] # define response variable
    # Remove labels from the dataset to define training set
    train = train[dt.columns.difference(dt.filter(like = 'risk').columns,sort=False)]
    # Remove categorical columns since they are only descriptive
    num_cols = train._get_numeric_data().columns
    to_drop = list(set(train.columns) - set(num_cols))
    to_drop.append("c40")
    train = train[train.columns.drop(to_drop)]
    # Define test set
    test = dt[~dt.index.isin(train.index)]
    test = test[test.columns.drop(to_drop)]
    test = test[test.columns.difference(test.filter(like = 'risk').columns,sort=False)]
    y_test = [random.randrange(4) for x in range(len(test))]
    return train, test, y_train, y_test

# Feature selection

## LASSO

In [4]:
from sklearn.linear_model import LassoCV

def lasso(X_train, y):
    lasso = LassoCV().fit(train, y)
    importance = np.abs(lasso.coef_)
    feature_names = np.array(train.columns)
    coef = pd.Series(lasso.coef_, index = train.columns)
    return coef[coef !=0]

In [5]:
risks = list(dt.filter(like='risk').columns)
for risk in risks:
    train, test, y_train, y_test = data_splitting(dt,risk)
    print(lasso(train,y_train))

population   -1.616826e-08
dtype: float64
population   -8.298024e-09
dtype: float64
Series([], dtype: float64)
Series([], dtype: float64)
population   -4.960183e-08
dtype: float64
population   -2.300692e-08
dtype: float64
Series([], dtype: float64)


In order to predict `risk0`, `risk1`, `risk4` and `risk5`, the LASSO method select only `population` as important regressor. The other risks cannot be predicted accurately on the basis of the given features.

## Boosting

In [6]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
model = xgb.XGBRegressor()

Feature selection using boosting regression with cross validation in order to tune the hyperparameters.

In [7]:
def boosting_reg(train, y_train, risk, predictions, accuracy_scores):
    reg_cv = GridSearchCV(model, {"colsample_bytree":[0.1,0.5,1.0],"min_child_weight":[1.0,1.2],
                            'max_depth': [7,9], 'n_estimators': [500], "alpha": [10,12,15]})
    reg_cv.fit(train,y_train)
    print(reg_cv.best_params_)
    gbm = xgb.XGBRegressor(**reg_cv.best_params_)
    gbm.fit(train,y_train)
    accuracy_scores[risk]=[gbm.score(train,y_train),0]
    sorted_idx = np.argsort(gbm.feature_importances_)[::-1]
    best_features = list()
    for index in sorted_idx:
        if gbm.feature_importances_[index] > 0:
            best_features.append(train.columns[index]) 
    return gbm, best_features[:10], accuracy_scores

In [8]:
def boosting_clas(gbm, test, y_test, risk, predictions, accuracy_scores):
    accuracy_scores[risk][1] = gbm.score(test,y_test)
    predictions[risk] = gbm.predict(test, iteration_range = (0, gbm.best_iteration))
    return predictions, accuracy_scores

In [ ]:
risks = list(dt.filter(like='risk').columns)
accuracy_scores = dict()
best_predictors = set()
predictions = dict()
for risk in risks:
    train, test, y_train, y_test = data_splitting(dt,risk)
    gbm, features, accuracy_scores = boosting_reg(train, y_train, risk, predictions, accuracy_scores)
    best_predictors.update(features)
    predictions, accuracy_scores = boosting_clas(gbm, test, y_test, risk, predictions, accuracy_scores)

{'alpha': 10, 'colsample_bytree': 0.5, 'max_depth': 9, 'min_child_weight': 1.0, 'n_estimators': 500}


In [ ]:
# xgb.plot_importance(xg_reg, max_num_features = 15)
# plt.show()

In [ ]:
fname = "boost_model.json"
# gbm.load_model(fname)

In [ ]:
best_predictors

In [ ]:
accuracy_scores

In [ ]:
get_back = ["city", "country", "country_code", "c40"]
to_drop = set(dt.columns) - best_predictors - set(get_back)
reduced_dataset = dt[dt.columns.drop(to_drop)]

In [ ]:
reduced_dataset.head()

In [ ]:
# reduced_dataset.to_csv('reduced_dataset.csv',index=False)

In [ ]:
# gbm.save_model(fname)